# IMPORT LIBRARIES AND DEFINE SETTINGS

In [ ]:
#Libraries to import
import os
import cv2 #OpenCV
import numpy as np
import pandas as pd
import csv

#User settings
filename = ('C:/Fill/Out/This/Path.avi') #File directory and video name
buttons = ["a", "l"]  # Keys, as many as you want
speed_factor = 1 # Multiplier of frame rate
start_frame = 9000 # Start the video at a specified frame
csv_filename = ('Data.csv') # name the output file
csv_filepath = ('C:/Fill/Out/This/Path') # specify folder to save output file
csv_filepath = os.path.join(csv_filepath, csv_filename) # concatenates the filepath with the filename. Don't change.

# PLAY VIDEO AND SCORE

Basic Controls:
Pause video = spacebar
Close video = q
Score exploration = press and hold the key assigned for a given object 

In [ ]:
# Open the video file and start playing it with user settings
video = cv2.VideoCapture(filename)
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
fps = int(video.get(cv2.CAP_PROP_FPS))
speed = round((500/fps) / speed_factor)
event_log = []

# Loop through every frame of the video
while video.isOpened():
    # Read the current frame
    ret, frame = video.read()
    if not ret:
        break
    log_row = []
    cv2.imshow('Current Frame', frame)
    # Check if the user is pressing and holding any of the keys that we are detecting
    button_press = cv2.waitKey(speed)
    if button_press == 32:
        cv2.waitKey()
    for button in buttons:
         # Store a 0 in an output array for every frame with no input from a given key
        success = 0
        # If a given key is pressed, a 1 is stored
        if button_press == ord(button):
            success = 1

        log_row.append(success)
    event_log.append(log_row)
    if cv2.waitKey(speed) == ord('q'):
        break
    
video.release()

# Close the video playback window
cv2.destroyAllWindows()


# EXPORT EXPLORATION TIMESERIES DATA

In [ ]:
# Save the array of input events as a .csv file. Data from each key are stored in columns from left to right in the same order
# as the keys are listed in the 'buttons' variable
with open(csv_filepath, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file, dialect='excel')
    for row in event_log:
        csv_writer.writerow(row)

# DISPLAY SUMMARY DATA

In [ ]:
#Calculates total times (s) for each key by summing all frames with key input and dividing by video frame rate
data = pd.DataFrame(event_log, columns = buttons)
key_totals = data.sum(axis=0)
key_times = pd.DataFrame(key_totals/fps)
print(key_times)